In [1]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [131]:
customers = pd.read_csv('recommend_1.csv')
transactions = pd.read_csv('trx_data.csv')

In [4]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [5]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [6]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [7]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [8]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

('Execution time:', 0.27, 'minutes')


In [9]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [10]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [11]:
data_dummy = create_data_dummy(data)

In [12]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
(df_matrix.shape)

(24429, 300)

In [14]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [16]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [27]:
train, test = train_test_split(data, test_size = .3)
print(train.shape, test.shape)

((93509, 3), (40076, 3))


In [28]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [29]:
train_data

customerId,productId,purchase_count
4024,246,1
17875,16,3
7170,13,1
8170,50,2
22024,55,2
16142,25,1
1261,75,2
2163,87,3
24039,164,2
2711,2,3


In [30]:
test_data

customerId,productId,purchase_count
13891,88,1
14423,246,1
1905,89,1
5339,178,1
13062,50,1
904,254,1
24319,38,5
22885,2,1
7104,151,1
22748,105,1


In [31]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [32]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [132]:
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [133]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 93509 observations with 22576 users and 300 items.

Data prepared in: 0.145693s

93509 observations to process; with 300 unique items.

In [134]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 128982

recommendations finished on 2000/62483 queries. users per second: 154571

recommendations finished on 3000/62483 queries. users per second: 162999

recommendations finished on 4000/62483 queries. users per second: 170249

recommendations finished on 5000/62483 queries. users per second: 175482

...

recommendations finished on 59000/62483 queries. users per second: 164524

recommendations finished on 60000/62483 queries. users per second: 163952

recommendations finished on 61000/62483 queries. users per second: 163071

recommendations finished on 62000/62483 queries. users per second: 161088

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|     0      |    132    | 3.32075471698 |  1   |
|     0      |     37    | 3.09865470852 |  2   |
|     0      |     27    | 3.02803738318 |  3   |
|     0      |     0     | 2.99461786868 |  4   |
|     0      |     34    | 2.98701298701 |  5   |
|     0      |    248    | 2.85365853659 |  6   |
|     0      |     3     | 2.76849642005 |  7   |
|     0      |     82    | 2.71153846154 |  8   |
|     0      |    110    | 2.59868421053 |  9   |
|     0      |     10    | 2.57239057239 |  10  |
|     1      |    132    | 3.32075471698 |  1   |
|     1      |     37    | 3.09865470852 |  2   |
|     1      |     27    | 3.02803738318 |  3   |
|     1      |     0     | 2.99461786868 |  4   |
|     1      |     34    | 2.98701298701 |  5   |
|     1      |    248    | 2.85365853659 |  6   |
|     1      |     3     | 2.76849642005 |  7   |


In [137]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [138]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [139]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23324 users and 300 items.

Data prepared in: 0.087207s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 157679

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    167    |  1.0  |  1   |
|    1553    |    259    |  1.0  |  2   |
|    1553    |    152    |  1.0  |  3   |
|    1553    |     44    |  1.0  |  4   |
|    1553    |    115    |  1.0  |  5   |
|    1553    |     2     |  1.0  |  6   |
|    1553    |     1     |  1.0  |  7   |
|    1553    |    190    |  1.0  |  8   |
|    1553    |    109    |  1.0  |  9   |
|    1553    |    183    |  1.0  |  10  |
|   20400    |    167    |  1.0  |  1   |
|   20400    |    259    |  1.0  |  2   |
|   20400    |    152    |  1.0  |  3   |
|   20400    |     44    |  1.0  |  4   |
|   20400    |    115    |  1.0  |  5   |
|   20400    |     2     |  1.0  |  6   |
|   20400    |     1     |  1.0  |  7   |
|   20400    |    190    |  1.0  |  8   |
|   20400    |    109    |  1.0  |  9   |
|   20400    |    183    |  1.0  |  10  |
|   19750    |    167    |  1.0  |

In [113]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23295 users and 300 items.

Data prepared in: 0.096533s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 82905

+------------+-----------+----------------+------+
| customerId | productId |     score      | rank |
+------------+-----------+----------------+------+
|    1553    |    226    | 0.776595744681 |  1   |
|    1553    |    247    | 0.325136612022 |  2   |
|    1553    |    230    | 0.324444444444 |  3   |
|    1553    |    125    | 0.258156028369 |  4   |
|    1553    |    294    | 0.254135338346 |  5   |
|    1553    |     83    | 0.236842105263 |  6   |
|    1553    |    276    | 0.236111111111 |  7   |
|    1553    |    213    | 0.233494363929 |  8   |
|    1553    |    248    | 0.232558139535 |  9   |
|    1553    |    165    | 0.230446927374 |  10  |
|    1553    |     72    | 0.225840336134 |  11  |
|    1553    |    204    | 0.224691358025 |  12  |
|    1553    |    155    | 0.220689655172 |  13  |
|    1553    |    129    | 0.215927750411 |  14  |
|    1553    |    128    | 0.213365539452 |  15  |
|    1553    |     32    | 0.210820895522 |  16  |
|    1553    |    171    | 0.20

In [140]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
132    3.320755
37     3.098655
27     3.028037
0      2.994618
34     2.987013
248    2.853659
3      2.768496
82     2.711538
110    2.598684
10     2.572391
230    2.533898
58     2.519481
32     2.515337
226    2.504202
129    2.493590
252    2.458333
87     2.447236
91     2.423645
54     2.392000
68     2.361386
Name: purchase_count, dtype: float64

In [115]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 93509 observations with 22576 users and 300 items.

Data prepared in: 0.087732s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.046189s

recommendations finished on 1000/1000 queries. users per second: 36905.8

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|    1553    |     5     | 0.0662034004927 |  1   |
|    1553    |     1     | 0.0642025619745 |  2   |
|    1553    |     2     | 0.0604064762592 |  3   |
|    1553    |     41    | 0.0507854223251 |  4   |
|    1553    |     33    | 0.0480920523405 |  5   |
|    1553    |     35    | 0.0480533838272 |  6   |
|    1553    |    269    | 0.0455253869295 |  7   |
|    1553    |     8     | 0.0421492606401 |  8   |
|    1553    |     14    | 0.0410356968641 |  9   |
|    1553    |     31    | 0.0384743213654 |  10  |
|    1553    |    170    | 0.0344718694687 |  11  |
|    1553    |     20    | 0.0341895222664 |  12  |
|    1553    |     68    | 0.0340424478054 |  13  |
|    1553    |     82    | 0.0325220674276 |  14  |
|    1553    |     59    | 0.0303804725409 |  15  |
|    1553    |    195    | 0.0302475839853 |  16  |
|    1553   

In [141]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23324 users and 300 items.

Data prepared in: 0.090493s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.037295s

recommendations finished on 1000/1000 queries. users per second: 80716.8

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|    1553    |     35    | 0.0922882755597 |  1   |
|    1553    |     1     | 0.0803789893786 |  2   |
|    1553    |     5     | 0.0718193650246 |  3   |
|    1553    |     2     |  0.067600051562 |  4   |
|    1553    |     60    | 0.0455016096433 |  5   |
|    1553    |     31    | 0.0415543317795 |  6   |
|    1553    |     11    | 0.0397038459778 |  7   |
|    1553    |     21    | 0.0389223694801 |  8   |
|    1553    |     33    | 0.0389115810394 |  9   |
|    1553    |    167    | 0.0370626052221 |  10  |
|   20400    |    246    | 0.0547626614571 |  1   |
|   20400    |     26    | 0.0501618981361 |  2   |
|   20400    |    113    | 0.0489296317101 |  3   |
|   20400    |    215    |  0.044035255909 |  4   |
|   20400    |     1     | 0.0411549806595 |  5   |
|   20400    |     74    | 0.0408044457436 |  6   |
|   20400   

In [142]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23295 users and 300 items.

Data prepared in: 0.193102s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.036761s

recommendations finished on 1000/1000 queries. users per second: 73507.8

+------------+-----------+------------------+------+
| customerId | productId |      score       | rank |
+------------+-----------+------------------+------+
|    1553    |     92    |       0.0        |  1   |
|    1553    |    235    |       0.0        |  2   |
|    1553    |     48    |       0.0        |  3   |
|    1553    |     7     |       0.0        |  4   |
|    1553    |    127    |       0.0        |  5   |
|    1553    |     37    |       0.0        |  6   |
|    1553    |     34    |       0.0        |  7   |
|    1553    |    290    |       0.0        |  8   |
|    1553    |    185    |       0.0        |  9   |
|    1553    |     97    |       0.0        |  10  |
|   20400    |    235    |       0.0        |  1   |
|   20400    |     48    |       0.0        |  2   |
|   20400    |     7     |       0.0        |  3   |
|   20400    |    127    |       0.0        |  4   |
|   20400    |     37    |       0.0        |  5   |
|   20400    |     34    |       0.0        | 

In [143]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 93509 observations with 22576 users and 300 items.

Data prepared in: 0.085903s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.052524s

recommendations finished on 1000/1000 queries. users per second: 42933.2

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|    1553    |    132    | 3.32004174112 |  1   |
|    1553    |     37    | 3.09705235685 |  2   |
|    1553    |     27    | 3.02803738318 |  3   |
|    1553    |     0     | 2.99005303205 |  4   |
|    1553    |     34    | 2.98701298701 |  5   |
|    1553    |    248    | 2.85266912928 |  6   |
|    1553    |     3     | 2.76849642005 |  7   |
|    1553    |     82    | 2.70897881457 |  8   |
|    1553    |    110    | 2.58310617899 |  9   |
|    1553    |     10    | 2.56994961318 |  10  |
|   20400    |    132    | 3.31090671836 |  1   |
|   20400    |     37    | 3.09865470852 |  2   |
|   20400    |     27    | 3.02803738318 |  3   |
|   20400    |     0     | 2.99461786868 |  4   |
|   20400    |     34    | 2.98701298701 |  5   |
|   20400    |    248    | 2.85114637962 |  6   |
|   20400    |     3     | 2.76849642005 |  7   |


In [145]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23324 users and 300 items.

Data prepared in: 0.093433s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.075711s

recommendations finished on 1000/1000 queries. users per second: 111557

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    167    |  0.0  |  1   |
|    1553    |    259    |  0.0  |  2   |
|    1553    |    152    |  0.0  |  3   |
|    1553    |     44    |  0.0  |  4   |
|    1553    |    115    |  0.0  |  5   |
|    1553    |     2     |  0.0  |  6   |
|    1553    |     1     |  0.0  |  7   |
|    1553    |    190    |  0.0  |  8   |
|    1553    |    109    |  0.0  |  9   |
|    1553    |    183    |  0.0  |  10  |
|   20400    |    167    |  0.0  |  1   |
|   20400    |    259    |  0.0  |  2   |
|   20400    |    152    |  0.0  |  3   |
|   20400    |     44    |  0.0  |  4   |
|   20400    |    115    |  0.0  |  5   |
|   20400    |     2     |  0.0  |  6   |
|   20400    |     1     |  0.0  |  7   |
|   20400    |    190    |  0.0  |  8   |
|   20400    |    109    |  0.0  |  9   |
|   20400    |    183    |  0.0  |  10  |
|   19750    |    167    |  0.0  |

In [146]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23295 users and 300 items.

Data prepared in: 0.185869s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.063959s

recommendations finished on 1000/1000 queries. users per second: 92225.4

+------------+-----------+----------------+------+
| customerId | productId |     score      | rank |
+------------+-----------+----------------+------+
|    1553    |    226    | 0.776285681319 |  1   |
|    1553    |    247    | 0.325136612022 |  2   |
|    1553    |    230    | 0.324025697973 |  3   |
|    1553    |    125    | 0.258049002269 |  4   |
|    1553    |    294    | 0.254135338346 |  5   |
|    1553    |     83    | 0.23618721711  |  6   |
|    1553    |    276    | 0.235875887341 |  7   |
|    1553    |    213    | 0.233096753265 |  8   |
|    1553    |    248    | 0.23241461155  |  9   |
|    1553    |    165    | 0.230345825976 |  10  |
|   20400    |    226    | 0.776595744681 |  1   |
|   20400    |    247    | 0.325136612022 |  2   |
|   20400    |    230    | 0.324444444444 |  3   |
|   20400    |    125    | 0.257798996547 |  4   |
|   20400    |    294    | 0.253805426637 |  5   |
|   20400    |     83    | 0.236842105263 |  6   |
|   20400    |    276    | 0.23

In [148]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [122]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/16910 queries. users per second: 78449.8

recommendations finished on 2000/16910 queries. users per second: 83309

recommendations finished on 3000/16910 queries. users per second: 85905.7

recommendations finished on 4000/16910 queries. users per second: 88335.3

...

recommendations finished on 13000/16910 queries. users per second: 59955.1

recommendations finished on 14000/16910 queries. users per second: 60724.1

recommendations finished on 15000/16910 queries. users per second: 61567

recommendations finished on 16000/16910 queries. users per second: 62106.3


Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.00112359550562 | 0.000736074737405 |
|   2    | 0.00348905972797 |  0.00334350430494 |
|   3    | 0.0036861817465  |  0.00511702132065 |
|   4    | 0.00824955647546 |  0.0148612480136  |
|   5    | 0.00768775872265 |  0.0170273357762  |
|   6    |  0.00670214863   |  0.0177060420492  |
|   7    | 0.00715552927262 |  0.0218656719777  |
|   8    | 0.00690419869899 |  0.0241803758034  |
|   9    | 0.00664958275839 |  0.0260398333673  |
|   10   | 0.0065819041987  |  0.0288824992762  |
+--------+------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 1.03505787021

Per User RMSE (best)
+------------+-------+-------------------+
| customerId | count |        rmse       |
+------------+-------+-------------------+
|   11000    |   1   | 0.000953827895127 |
+--

In [149]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13840 queries. users per second: 78296.3

recommendations finished on 2000/13840 queries. users per second: 84164.5

recommendations finished on 3000/13840 queries. users per second: 86019

recommendations finished on 4000/13840 queries. users per second: 89843.2

...

recommendations finished on 10000/13840 queries. users per second: 78373

recommendations finished on 11000/13840 queries. users per second: 78332

recommendations finished on 12000/13840 queries. users per second: 77857.4

recommendations finished on 13000/13840 queries. users per second: 78377.5


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.00513005780347 | 0.00260069991826 |
|   2    | 0.00892341040462 | 0.00898059504061 |
|   3    |  0.010549132948  | 0.0155102399585  |
|   4    | 0.0103865606936  | 0.0200845701723  |
|   5    | 0.0106791907514  | 0.0267294392323  |
|   6    | 0.0106815992293  | 0.0325194777102  |
|   7    | 0.00984723369116 | 0.0347045692323  |
|   8    | 0.00922145953757 | 0.0370391804705  |
|   9    | 0.00887925497752 | 0.0401342182603  |
|   10   | 0.00852601156069 | 0.0428764114396  |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|   16436    |   2   | 1.0  |
+------------+-------+------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---

In [124]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13898 queries. users per second: 70254.3

recommendations finished on 2000/13898 queries. users per second: 78373

recommendations finished on 3000/13898 queries. users per second: 83521.3

recommendations finished on 4000/13898 queries. users per second: 86538.9

...

recommendations finished on 10000/13898 queries. users per second: 61918.4

recommendations finished on 11000/13898 queries. users per second: 62060.3

recommendations finished on 12000/13898 queries. users per second: 57843.3

recommendations finished on 13000/13898 queries. users per second: 58294.4


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.00280615915959 | 0.0012729960834  |
|   2    | 0.00266225356166 | 0.0028315041369  |
|   3    | 0.0025903007627  | 0.00455237524545 |
|   4    | 0.00244639516477 | 0.0056017725551  |
|   5    | 0.00241761404519 | 0.00710013029177 |
|   6    | 0.0022545210342  | 0.00789221068703 |
|   7    | 0.00235388442325 | 0.00960607586516 |
|   8    | 0.00260828896244 | 0.0117619784762  |
|   9    | 0.00249436369741 | 0.0123955013467  |
|   10   | 0.00273420636063 | 0.0149415067547  |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.131160648064

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|   26590    |   1   | 0.0  |
+------------+-------+------+
[1 rows x 3 columns]


Per User RMSE (

In [150]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                           user_id=user_id, 
                                           item_id=item_id, 
                                           target='purchase_dummy', similarity_type='cosine'
                                       )
#cos_dummy = final_model(train_data_dummy, 'cosine', user_id, item_id, 'purchase_dummy', users_to_recommend, n_rec, n_display)
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.097427s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.036573s

recommendations finished on 1000/1000 queries. users per second: 47078.8

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|    1553    |     2     |  0.123247849941 |  1   |
|    1553    |     35    |  0.104471671581 |  2   |
|    1553    |     5     | 0.0906752586365 |  3   |
|    1553    |     1     | 0.0867011427879 |  4   |
|    1553    |     17    | 0.0765981435776 |  5   |
|    1553    |     21    | 0.0749170780182 |  6   |
|    1553    |     33    | 0.0668614387512 |  7   |
|    1553    |     47    | 0.0605887889862 |  8   |
|    1553    |     61    | 0.0603173971176 |  9   |
|    1553    |     15    | 0.0594960331917 |  10  |
|   20400    |     26    | 0.0581226944923 |  1   |
|   20400    |     6     | 0.0536174178123 |  2   |
|   20400    |    113    | 0.0531278848648 |  3   |
|   20400    |     1     | 0.0521045923233 |  4   |
|   20400    |     15    | 0.0476838946342 |  5   |
|   20400    |     27    | 0.0446733832359 |  6   |
|   20400   

In [152]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.123248,1
1,1553,35,0.104472,2
2,1553,5,0.090675,3
3,1553,1,0.086701,4
4,1553,17,0.076598,5


In [153]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')
df_output.head()

,recommendedProducts
customerId,
4,2|1|13|36|20|25|216|61|33|157
11,2|5|15|11|7|33|13|17|21|0
12,20|2|1|15|12|25|82|5|36|14
16,1|5|21|15|17|38|13|35|36|44
21,38|36|48|79|2|1|15|13|44|63


In [154]:
#def create_output( users_to_recommend,  print_csv=True):
    #recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    #df_rec = recomendation.to_dataframe()
    #df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
#df_output = df_output[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')
    #if print_csv:
df_output.to_csv('output/option1_recommendation.csv')
print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    #return df_output


An output file can be found in 'output' folder with name 'option1_recommendation.csv'


In [155]:
#users_to_recommend = list(customers[user_id])
#df_output = create_output( users_to_recommend, print_csv=True)
#print(df_output.shape)
df_output.head()

,recommendedProducts
customerId,
4,2|1|13|36|20|25|216|61|33|157
11,2|5|15|11|7|33|13|17|21|0
12,20|2|1|15|12|25|82|5|36|14
16,1|5|21|15|17|38|13|35|36|44
21,38|36|48|79|2|1|15|13|44|63
